CONNECT TO SQLITE

importing the CSV into a clean database, creating raw, cleaned, and normalized tables (raw_data, clean_data, customers, locations, rides), and validating everything step by step so the system is now stable and ready for analysis.

In [1]:
import sqlite3
import pandas as pd

db_path = r"C:\Users\Nimit\OneDrive\Desktop\Ola Project\db\ola_analytics.db"
conn = sqlite3.connect(db_path)

print("Connected to SQLite")


Connected to SQLite


In [2]:
csv_path = r"C:\Users\Nimit\OneDrive\Desktop\Ola Project\data\ncr_ride_bookings.csv"

df = pd.read_csv(csv_path)

df.to_sql(
    name="raw_data",
    con=conn,
    if_exists="replace",
    index=False,
    chunksize=10000
)

print("CSV imported")


CSV imported


In [3]:
pd.read_sql("SELECT COUNT(*) AS rows FROM raw_data;", conn)


,rows
0,150000


CREATE clean_data

In [4]:
conn.execute("DROP TABLE IF EXISTS clean_data;")
conn.execute("""
CREATE TABLE clean_data AS
SELECT * FROM raw_data;
""")
conn.commit()

print("clean_data created")


clean_data created


RENAME COLUMNS

In [5]:
rename_queries = [
    'ALTER TABLE clean_data RENAME COLUMN "Booking ID" TO booking_id;',
    'ALTER TABLE clean_data RENAME COLUMN "Booking Status" TO booking_status;',
    'ALTER TABLE clean_data RENAME COLUMN "Customer ID" TO customer_id;',
    'ALTER TABLE clean_data RENAME COLUMN "Vehicle Type" TO vehicle_type;',
    'ALTER TABLE clean_data RENAME COLUMN "Pickup Location" TO pickup_location;',
    'ALTER TABLE clean_data RENAME COLUMN "Drop Location" TO drop_location;',
    'ALTER TABLE clean_data RENAME COLUMN "Booking Value" TO booking_value;',
    'ALTER TABLE clean_data RENAME COLUMN "Ride Distance" TO ride_distance;',
    'ALTER TABLE clean_data RENAME COLUMN "Driver Ratings" TO driver_ratings;',
    'ALTER TABLE clean_data RENAME COLUMN "Customer Rating" TO customer_rating;',
    'ALTER TABLE clean_data RENAME COLUMN "Payment Method" TO payment_method;'
]

for q in rename_queries:
    conn.execute(q)

conn.commit()
print("Columns renamed")


Columns renamed


BASIC CLEANING

In [6]:
conn.execute("""
DELETE FROM clean_data
WHERE booking_id IS NULL
   OR customer_id IS NULL
   OR Date IS NULL;
""")

conn.execute("""
UPDATE clean_data
SET
booking_status = LOWER(TRIM(booking_status)),
vehicle_type = LOWER(TRIM(vehicle_type)),
pickup_location = LOWER(TRIM(pickup_location)),
drop_location = LOWER(TRIM(drop_location)),
payment_method = LOWER(TRIM(payment_method));
""")

conn.commit()
print("Basic cleaning done")


Basic cleaning done


CREATE customers TABLE

In [8]:
conn.execute("DROP TABLE IF EXISTS customers;")
conn.execute("""
CREATE TABLE customers AS
SELECT DISTINCT customer_id
FROM clean_data;
""")
conn.commit()


CREATE locations TABLE

In [9]:
conn.execute("DROP TABLE IF EXISTS locations;")
conn.execute("""
CREATE TABLE locations AS
SELECT DISTINCT pickup_location AS location_name FROM clean_data
UNION
SELECT DISTINCT drop_location FROM clean_data;
""")
conn.commit()


CREATE rides TABLE (FACT TABLE)

In [10]:
conn.execute("DROP TABLE IF EXISTS rides;")
conn.execute("""
CREATE TABLE rides AS
SELECT
    booking_id,
    Date AS booking_date,
    Time AS booking_time,
    customer_id,
    pickup_location,
    drop_location,
    vehicle_type,
    booking_status,
    ride_distance,
    booking_value,
    driver_ratings,
    customer_rating,
    payment_method
FROM clean_data;
""")
conn.commit()


FINAL SANITY CHECKS

In [11]:
pd.read_sql("SELECT COUNT(*) FROM rides;", conn)


,COUNT(*)
0,150000


In [12]:
pd.read_sql("""
SELECT COUNT(*) AS total,
       COUNT(DISTINCT booking_id) AS unique_ids
FROM rides;
""", conn)


,total,unique_ids
0,150000,148767


In [13]:
conn.close()
print("Database build completed successfully")


Database build completed successfully
